In [1]:
import numpy as np
from ur_analytic_ik import ur5e

np.set_printoptions(precision=4, suppress=True) # Prettier printing


In [2]:
with open("ur5e_poses.txt") as f:
    lines = f.readlines()


In [3]:
joint_config_pose_pairs = []
for line in lines:
    if len(line) == 1:
        continue
    joint_config, pose = line.split(";")[0], line.split(";")[1]
    joint_config = joint_config.split(",")
    # strip the brackets
    joint_config[0] = joint_config[0][1:]
    joint_config[-1] = joint_config[-1][:-1]
    joint_config = [float(j) for j in joint_config]
    
    pose = pose.split(",")
    # remove all brackets
    for i in range(len(pose)):
        pose[i] = pose[i].replace("[", "").replace("]", "")
    pose = [float(p) for p in pose]
    pose = np.array(pose).reshape(4, 4)
    joint_config_pose_pairs.append((joint_config, pose))


In [4]:
fk_poses = []

for joint_config, pose in joint_config_pose_pairs:
    fk_pose = ur5e.forward_kinematics(*joint_config)
    fk_poses.append(fk_pose)
    distance = np.linalg.norm(fk_pose[:3,3] - pose[:3,3])
    print(f"fk pose vs real pose with distance {distance} m")



fk pose vs real pose with distance 0.0018118633601494816 m
fk pose vs real pose with distance 0.001244811756654254 m
fk pose vs real pose with distance 0.0016960076410719273 m
fk pose vs real pose with distance 0.001823231630988334 m
fk pose vs real pose with distance 0.0013480485905176857 m
fk pose vs real pose with distance 0.001334366907552316 m
fk pose vs real pose with distance 0.001429664226362876 m
fk pose vs real pose with distance 0.0014651598985280685 m
fk pose vs real pose with distance 0.0014481558396959072 m
fk pose vs real pose with distance 0.0018434954707430456 m


In [13]:
for joint_config,pose in joint_config_pose_pairs:
    joint_config_ik = ur5e.inverse_kinematics_closest(pose,*joint_config)
    if len(joint_config_ik) == 0:
        print("no ik solution found")
        continue
    joint_config_ik = np.array(joint_config_ik[0])

    distance = np.linalg.norm(joint_config-joint_config_ik)
    print(f"joint config vs ik joint config with distance {distance} rad")

joint config vs ik joint config with distance 0.007135091262535225 rad
joint config vs ik joint config with distance 0.0068736539252271835 rad
no ik solution found
joint config vs ik joint config with distance 0.00648563672138501 rad
joint config vs ik joint config with distance 0.011152488236170929 rad
joint config vs ik joint config with distance 0.01071535318145833 rad
joint config vs ik joint config with distance 0.008245401296536689 rad
joint config vs ik joint config with distance 0.008531492436559675 rad
joint config vs ik joint config with distance 0.00854205339004058 rad
joint config vs ik joint config with distance 0.007467297721403651 rad
